In [ ]:
baizer::pkglib(baizer, tidyverse, ggsci, ggbeeswarm, plutor, ComplexHeatmap, plotly, ggplotify, echarts4r)

In [ ]:
sample <- snakemake@wildcards[['sample']]

# replace default configs with sample configs
config <- replace_item(snakemake@config, snakemake@config[[sample]])
Lflt_mode <- names(config[['flt_mode']])

In [ ]:
filter_dir <- snakemake@input[['filter_dir']]
stat_dir <- snakemake@input[['stat_dir']]

In [ ]:
flt_mode <- Lflt_mode[1]

In [ ]:
TBkeep <- read_csv(str_glue('{filter_dir}/{flt_mode}_keep.csv'))
TBflt <- read_csv(str_glue('{filter_dir}/{flt_mode}_flt.csv'))

In [ ]:
Vcol <- c('#E64B35FF', '#4DBBD5FF', '#00A087FF', '#3C5488FF', '#F39B7FFF', '#8491B4FF', '#91D1C2FF', '#DC0000FF', '#7E6148FF', '#B09C85FF', '#1F77B4FF', '#FF7F0EFF', '#2CA02CFF', '#D62728FF', '#9467BDFF', '#8C564BFF', '#E377C2FF', '#7F7F7FFF', '#BCBD22FF', '#17BECFFF', '#AEC7E8FF', '#FFBB78FF', '#98DF8AFF', '#FF9896FF', '#C5B0D5FF', '#C49C94FF', '#F7B6D2FF', '#C7C7C7FF', '#DBDB8DFF', '#9EDAE5FF')
grey_col <- '#f5f5f5'

In [ ]:
# filter

In [ ]:
TBkeep <- TBkeep %>% c2r('cell') %>% seriate_df %>% r2c('cell')

In [ ]:
TBkeep_longer <- TBkeep %>% pivot_longer(-cell, names_to='term', values_to='value')

In [ ]:
TBkeep_longer %>% 
  mutate(term=str_replace(term, '_keep$', '')) %>%
  e_charts(term) %>% 
  e_heatmap(cell, value) %>%
  e_visual_map(value) %>%
  e_title("Filter") %>%
  e_x_axis(axisLabel=list(interval=0, rotate=45)) %>%
  e_y_axis(show=FALSE)

In [ ]:
as_tibble(TBkeep %>% select(-cell) %>% apply(2, sum), rownames='term') %>%
  mutate(term=str_replace(term, '_keep$', '')) %>%
  e_charts(term) %>%
  e_bar(value) %>%
  e_x_axis(axisLabel=list(interval=0, rotate=45)) %>%
  e_title("Filter") 

In [ ]:
# FB

In [ ]:
## HT

In [ ]:
if (length(config[[sample]][['id2seq']][['HT']]) > 0) {
    TBfb_ht <- TBflt %>% select(cell, matches('^HT_'), sum_HT)

    e1 <- TBfb_ht %>% 
        group_by(HT_group) %>% 
        e_charts(sum_HT) %>% 
        e_title("Hashtag") %>%
        e_scatter(HT_ratio, symbol_size=10) %>%
        e_tooltip(trigger = 'axis')

    e2 <- TBfb_ht %>% count(HT_group) %>%
      e_charts(HT_group) |> 
      e_pie(n, radius = c("50%", "70%")) |> 
      e_title("Hashtag group")
    
    e_arrange(e1, e2)
}

In [ ]:
## BD

In [ ]:
TBfb_bd <- TBflt %>% select(cell, matches('^BD_[^#]+$'), -matches('_ratio$'), -BD_type)

In [ ]:
TBfb_bd_longer <- TBfb_bd %>% c2r('cell') %>% seriate_df %>% r2c('cell') %>% 
    pivot_longer(-cell, names_to='BD', values_to='value')

In [ ]:
TBfb_bd_longer %>% 
  e_charts(BD) %>% 
  e_heatmap(cell, value) %>%
  e_visual_map(value) %>%
  e_title("Binding") %>%
  e_x_axis(axisLabel=list(interval=0, rotate=45)) %>%
  e_y_axis(show=FALSE)

In [ ]:
TBfb_bd <- TBflt %>% select(cell, matches('^BD_.+_ratio$'))

In [ ]:
TBfb_bd_longer <- TBfb_bd %>% c2r('cell') %>% 
    seriate_df %>% r2c('cell') %>% 
    pivot_longer(-cell, names_to='BD', values_to='value')

In [ ]:
TBfb_bd_longer %>% 
  e_charts(BD) %>% 
  e_heatmap(cell, value) %>%
  e_visual_map(value) %>%
  e_title("Binding ratio") %>%
  e_x_axis(axisLabel=list(interval=0, rotate=45)) %>%
  e_y_axis(show=FALSE)

In [ ]:
js <- '
formatter(param) {
  return param.name + " (" + param.percent * 2 + "%)";
}
'
js <- structure(js, class = c("JS_EVAL"))

In [ ]:
TBflt %>% count(BD_type) %>%
  e_charts(BD_type) %>%
  e_pie(n, radius = c("50%", "70%"), label=list(show=TRUE, formatter='{b}({c}, {d}%)')) %>%
  e_title("Binding type")

In [ ]:
# VDJB

In [ ]:
TBp <- TBflt %>% count(clone=clone_cellranger) 
Nclones <- TBp %>% filter(n > 1) %>% nrow
Nclones_percent <- float_to_percent(Nclones / nrow(TBflt))

TBp %>%
  e_charts(clone) %>%
  e_pie(n, radius = c("50%", "70%"), label=list(show=TRUE, position='center', formatter=Nclones_percent), 
        itemStyle=list(borderColor='grey')) %>%
  e_title("Class") %>%
  e_legend(show=FALSE) %>%
  pe_color(gradient_color(Vcol, n=seq_len(Nclones)), default=grey_col)

In [ ]:
TBflt %>% count(class) %>% filter(!is.na(class)) %>%
  e_charts(class) %>%
  e_pie(n, radius = c("50%", "70%")) %>%
  e_title("Class")

In [ ]:
TBflt %>% select(class, v_domain_shm_ratio_H) %>%
  group_by(class) %>%
  e_chart() %>%
  e_density(v_domain_shm_ratio_H, areaStyle = list(opacity = .4), smooth = TRUE) %>%
  e_tooltip() %>%
  e_title("Heavy Chain SHM")

In [ ]:
TBflt %>% select(class, v_domain_shm_ratio_L) %>%
  group_by(class) %>%
  e_chart() %>%
  e_density(v_domain_shm_ratio_L, areaStyle = list(opacity = .4), smooth = TRUE) %>%
  e_tooltip() %>%
  e_title("Light Chain SHM")

In [ ]:
# out

In [ ]:
out <- snakemake@output[['visualize_touch']]
dir.create(dirname(out), recursive = TRUE)
F <- file(out, "w" )
writeLines('')
close(F)